# Fixing the SN data frame

In [31]:
import load_data
import preprocess_spectra
import pandas
from importlib import reload
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
cmap = get_cmap('viridis')
import numpy
import specplotlib
from sklearn.linear_model import LinearRegression
%matplotlib inline

In [32]:
# Add constitution to df

SN_df_w_salt_w_class = pandas.read_csv('SN_df_w_salt_w_class.csv')
SN_df_w_salt_w_class = SN_df_w_salt_w_class.set_index("#SN", drop=False)
SN_df_w_salt_w_class.index = [s.lower() for s in SN_df_w_salt_w_class.index]

const_salt = pandas.read_csv('const_param.txt', delimiter='\s+', header=None, skiprows=5)
const_salt.columns = ["SNIa_salt", "z_cmb_salt", "z_err_salt", "m_B_salt", "m_B_err_salt", "s_salt", "s_err_salt", "c_salt", "c_err_salt", "mu_salt", "mu_err_salt"]
const_salt = const_salt.set_index('SNIa_salt', drop=False)
const_salt.index = [s.lower() for s in const_salt.index]

index_tmp = const_salt.index
index_new = []
for s in index_tmp:
    if 'sn' in s:
        s = s[2:]
        if (s[0] == '0') or (s[0] == '1'):
            s = '20' + s
        else:
            s = '19' + s
    index_new += [s]
const_salt.index = index_new

SN_df_w_salt_w_class = SN_df_w_salt_w_class.merge(const_salt, left_index=True, right_index=True, how='left')
SN_df_w_salt_w_class.shape


(229, 50)

In [40]:
# Fix back index in SN_df and delete unnecesary columns
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'Unnamed: 0':'idx'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.set_index('idx', drop=False)
SN_df_w_salt_w_class = SN_df_w_salt_w_class.drop(['z_distmod', 'alpha', 'beta'], axis=1)

In [41]:
# Rename all columns

#from spectra (‘/Users/natalie/git/SNIa/data/cfaspec_snIa/cfasnIa_param.dat’):
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'#SN':'SN'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'zhel':'zhel_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'tmax(B)':'tmax(B)_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'+/-_x':'tmax(B)_err_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'ref.':'ref_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'Dm15':'Dm15_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'+/-.1_x':'Dm15_err_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'ref..1':'ref1_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'M_B':'M_B_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'+/-.2_x':'M_B_err_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'B-V':'B-V_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'+/-.3_x':'B-V_err_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'Bm-Vm':'Bm-Vm_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'+/-.4_x':'Bm-Vm_err_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'Phot.':'Phot_ref_spec'})

#from spectra (/Users/natalie/git/SNIa/data/cfaspec_snIa/branchwangclass.dat)
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'EW5972':'EW5972_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'EW6355':'EW6355_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'v6355':'v6355_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'Phase':'Phase_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'BClass':'BClass_spec'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'WClass':'WClass_spec'})

#the rest?
# 'RA', 'DEC','z_CMB', '+/-_y', 'm_B', '+/-.1_y', 'x_1', '+/-.2_y', 'c', '+/-.3_y','mu', '+/-.4_y', 'z_distmod', 'alpha', 'beta',
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'RA':'RA'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'DEC':'DEC'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'z_CMB_salt2':'z_CMB_salt2'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'+/-_y':'z_CMB_salt2_err'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'m_B':'m_B_salt2'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'+/-.1_y':'m_B_salt2_err'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'x_1':'x_1_salt2'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'+/-.2_y':'x_1_salt2_err'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'c':'c_salt2'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'+/-.3_y':'c_salt2_err'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'mu':'mu_salt2'})
SN_df_w_salt_w_class = SN_df_w_salt_w_class.rename(columns = {'+/-.4_y':'mu_salt2_err'})

#From constitution: already good and all end with ''***_salt'


## checks

In [42]:
SN_df_w_salt_w_class.columns

Index(['idx', 'SN', 'zhel_spec', 'tmax(B)_spec', 'tmax(B)_err_spec',
       'ref_spec', 'Dm15_spec', 'Dm15_err_spec', 'ref1_spec', 'M_B_spec',
       'M_B_err_spec', 'B-V_spec', 'B-V_err_spec', 'Bm-Vm_spec',
       'Bm-Vm_err_spec', 'Phot_ref_spec', 'ref..2', 'n_spec', 'RA', 'DEC',
       'z_CMB', 'z_CMB_salt2_err', 'm_B_salt2', 'm_B_salt2_err', 'x_1_salt2',
       'x_1_salt2_err', 'c_salt2', 'c_salt2_err', 'mu_salt2', 'mu_salt2_err',
       'EW5972_spec', 'EW6355_spec', 'v6355_spec', 'Phase_spec', 'BClass_spec',
       'WClass_spec', 'SNIa_salt', 'z_cmb_salt', 'z_err_salt', 'm_B_salt',
       'm_B_err_salt', 's_salt', 's_err_salt', 'c_salt', 'c_err_salt',
       'mu_salt', 'mu_err_salt'],
      dtype='object')

In [43]:
const_salt.head(3)

,SNIa_salt,z_cmb_salt,z_err_salt,m_B_salt,m_B_err_salt,s_salt,s_err_salt,c_salt,c_err_salt,mu_salt,mu_err_salt
1993ag,1993ag,0.050,0.001,17.79,0.108,0.91,0.02,0.09,0.02,36.910,0.135
1993o,1993o,0.053,0.001,17.61,0.111,0.90,0.01,-0.01,0.02,36.977,0.134
1993h,1993h,0.025,0.001,16.74,0.145,0.68,0.01,0.21,0.01,35.264,0.186


In [44]:
SN_df_w_salt_w_class.head(3)

,idx,SN,zhel_spec,tmax(B)_spec,tmax(B)_err_spec,ref_spec,Dm15_spec,Dm15_err_spec,ref1_spec,M_B_spec,...,z_cmb_salt,z_err_salt,m_B_salt,m_B_err_salt,s_salt,s_err_salt,c_salt,c_err_salt,mu_salt,mu_err_salt
idx,,,,,,,,,,,,,,,,,,,,,
0,0,1994D,0.0029,49432.5,0.1,"B,M,SA,SN",1.37,0.03,B,-99.99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1994M,0.0232,49474.5,0.9,"B,M,SA,SN",1.26,0.06,B,-19.21,...,0.024,0.001,16.30,0.159,0.78,0.02,0.11,0.02,35.207,0.205
2,2,1994S,0.0151,49518.3,0.3,"B,M,SA,SN",0.94,0.06,B,-19.55,...,0.015,0.001,14.77,0.196,1.09,0.05,-0.01,0.02,34.372,0.280


In [45]:
print('len const_salt: %s' % len(const_salt['SNIa_salt'].unique()))
print('len SN_salt_df In SN_df: %s' % (len(SN_df_w_salt_w_class)))
print('len const_salt In SN_df: %s' % numpy.isfinite(SN_df_w_salt_w_class['s_salt'].values).sum())
print('len spectra data in SN_df: %s' % (SN_df_w_salt_w_class['M_B_spec'].values>-75).sum())

len const_salt: 397
len SN_salt_df In SN_df: 229
len const_salt In SN_df: 92
len spectra data in SN_df: 103


In [46]:
const_salt['SNIa_salt'].unique()

array(['1993ag', '1993o', '1993h', '1993b', '1992bs', '1992br', '1992bp',
       '1992bo', '1992bl', '1992bh', '1992bc', '1992aq', '1992ag',
       '1992ae', '1992p', '1990af', '1990o', '2001cz', '2001cn', '2001ba',
       '2000ca', '2000bh', '1999gp', '1993ac', '1994m', '1994s', '1994t',
       '1995ac', '1995ak', '1996c', '1996ab', '1996bl', '1996bv',
       '1996bo', '2000fa', '2000dk', '2000cn', '2000cf', '1999gd',
       '1999ek', '1999cc', '1998eg', '1998ef', '1998dx', '1998co',
       '1998ab', '1998v', '1997dg', '1997y', '1999aa', '1999ao', '1999ar',
       '1999aw', '1999bi', '1999bm', '1999bn', '1999bp', '1996h', '1996i',
       '1996j', '1996k', '1996u', '1995ao', '1995ap', '1996t', '1997ce',
       '1997cj', '1997ck', '1995k', '1997ap', '1997am', '1997aj',
       '1997ai', '1997af', '1997ac', '1997r', '1997p', '1997o', '1997h',
       '1997g', '1997f', '1996cn', '1996cm', '1996cl', '1996ck', '1996ci',
       '1996cg', '1996cf', '1995ba', '1995az', '1995ay', '1995ax',
      

In [ ]:
############## still missing all the ones in the middle and how to get all the const in - see:
# https://archive.stsci.edu/prepds/ps1cosmo/scolnic_datatable.html 
# https://arxiv.org/pdf/1611.07042.pdf
# https://arxiv.org/pdf/1710.00845.pdf

In [47]:
SN_df_w_salt_w_class.to_csv('good_df.csv', index=False)